In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
from datasets import load_dataset, load_metric

raw_datasets = load_dataset("glue" , "cola")

In [3]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 8551
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1043
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1063
    })
})

In [4]:
from transformers import AutoTokenizer

model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [5]:
def preprocess_function(examples):
    return tokenizer(examples["sentence"], truncation=True)

tokenized_datasets = raw_datasets.map(preprocess_function, batched = True)

In [6]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)

In [7]:
import numpy as np
from datasets import load_metric

metric = load_metric("glue" , "cola")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

C:\Users\Emincan\AppData\Local\Temp\ipykernel_25672\2822989607.py:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("glue" , "cola")


In [8]:
from transformers import TrainingArguments

args = TrainingArguments(
    model_checkpoint,
    evaluation_strategy= "epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5, 
    weight_decay= 0.02,
    push_to_hub= True,
    # hub_model_id= "organization/other-name"
)

In [9]:
from transformers import Trainer

trainer = Trainer(
    model,
    args,
    train_dataset= tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics= compute_metrics,
    tokenizer= tokenizer
)

trainer.train()

  0%|          | 0/5345 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 0.6333, 'learning_rate': 4.5322731524789526e-05, 'epoch': 0.47}
{'loss': 0.6138, 'learning_rate': 4.064546304957905e-05, 'epoch': 0.94}


  0%|          | 0/131 [00:00<?, ?it/s]

{'eval_loss': 0.6181532740592957, 'eval_matthews_correlation': 0.0, 'eval_runtime': 0.9872, 'eval_samples_per_second': 1056.548, 'eval_steps_per_second': 132.702, 'epoch': 1.0}
{'loss': 0.6234, 'learning_rate': 3.596819457436857e-05, 'epoch': 1.4}
{'loss': 0.6095, 'learning_rate': 3.129092609915809e-05, 'epoch': 1.87}


...

{'eval_loss': 0.6182088255882263, 'eval_matthews_correlation': 0.0, 'eval_runtime': 0.994, 'eval_samples_per_second': 1049.271, 'eval_steps_per_second': 131.788, 'epoch': 3.0}
{'loss': 0.6149, 'learning_rate': 1.725912067352666e-05, 'epoch': 3.27}
{'loss': 0.6133, 'learning_rate': 1.2581852198316186e-05, 'epoch': 3.74}


  0%|          | 0/131 [00:00<?, ?it/s]

{'eval_loss': 0.6180768609046936, 'eval_matthews_correlation': 0.0, 'eval_runtime': 0.9867, 'eval_samples_per_second': 1057.075, 'eval_steps_per_second': 132.768, 'epoch': 4.0}
{'loss': 0.6151, 'learning_rate': 7.904583723105706e-06, 'epoch': 4.21}
{'loss': 0.61, 'learning_rate': 3.2273152478952295e-06, 'epoch': 4.68}


  0%|          | 0/131 [00:00<?, ?it/s]

{'eval_loss': 0.6208435893058777, 'eval_matthews_correlation': 0.0, 'eval_runtime': 0.9793, 'eval_samples_per_second': 1065.062, 'eval_steps_per_second': 133.771, 'epoch': 5.0}
{'train_runtime': 198.9702, 'train_samples_per_second': 214.881, 'train_steps_per_second': 26.863, 'train_loss': 0.6149352028171408, 'epoch': 5.0}


In [ ]:
trainer.push_to_hub("End of training6")
# trainer.push_to_hub("End of training" , create_pr= True)
# trainer.push_to_hub(commit_message="End of training", blocking=True)

# Pushing Components individually

In [ ]:
repo_name = "bert-base-cased"

model.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)

# Labels

In [ ]:
label_names = raw_datasets["train"].features["label"].names
label_names

In [ ]:
model.config.id2label = {str(i): lbl for i, lbl in enumerate(label_names)}
model.config.label2id = {lbl:str(i) for i, lbl in enumerate(label_names)}

In [ ]:
repo_name = "bert-base-cased"
model.config.push_to_hub(repo_name)

In [ ]:
# Done